# 5. Evolve with SUPG, timedep

In [1]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])

# Iliescu Exact Solution
c = 16.0*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
ue = c*h*(0.5+sym.atan(g)/sym.pi)

# ------------------------------------------ #

ue = sym.simplify(ue)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = sym.diff(ue,t)
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')

# print('u_code = ' + u_code + '\n')
# print('f_code = ' + f_code)

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

In [2]:
def evolve_t(nx, dt, T, u_code, f_code, sigma, mu, velocity):
    folder = 'FEFfigs/'
    degree = 2
    
    t = 0.0
    u_exact = Expression(u_code, degree = degree, t = t)
    f = Expression(f_code, degree = degree, t = t)

    mesh = UnitSquareMesh(nx,nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_code, degree = degree, t = t)
    
    def boundary(x, on_boundary):
        return on_boundary

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    #u_n = Function(Q)
    u_n = interpolate(u_D, Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = (u - u_n)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)

    # SUPG stabilization terms
    h = CellDiameter(mesh)

#     r = u - u_n + dt*(- mu*div(grad(u)) + dot(velocity, grad(u)) + sigma*u - f)
#     vnorm = sqrt(dot(velocity, velocity))
#     F += (h/(2.0*vnorm))*dot(velocity, grad(v))*r*dx

    # based on paper's definition of residual and stabilization term
    Lt = -mu*div(grad(u)) + dot(velocity, grad(u)) + (sigma+1.0/dt)*u 
    ft = u_n/dt + f
    r = ft - Lt
    vnorm = sqrt(dot(velocity, velocity))
    F -= dt*(h/(2.0*vnorm))*dot(velocity, grad(v))*r*dx    
    
    
    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)
    
    # Outputting files
    out_file_uexact = File(folder+"evolve_u_exact_"+str(nx)+".pvd") 
    out_file_ubar = File(folder+"SUPG_u_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    
    u_.rename('u','u')
    
    # Save t = 0.0
    out_file_uexact << (ue, float(t))
    out_file_ubar << (u_, float(t))

    while t - T + dt < DOLFIN_EPS:
        # Step 1 Solve on Coarse Grid
        t += dt
        
        u_.rename('u','u')
        u_exact.rename('u','u')
        
        u_D.t = t
        f.t = t
        u_exact.t = t
        
        b = assemble(L)
        bc = DirichletBC(Q, u_D, boundary)
        bc.apply(A1)
        bc.apply(b)
        
        solve(A1, u_.vector(), b)
        progress.update(t / T)
        out_file_uexact << (ue, float(t))
        out_file_ubar << (u_, float(t))
        
        u_n.assign(u_)
        
    L2, H1 = compute_errors(u_exact, u_, t, mesh)
    maxval, minval = compute_extrema(u_, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)

SUPG

* u_D = u_exact

dt=0.01
T=0.01
print("nx L2 H1 maxval minval")
for nx in [25, 50, 100, 200, 400]:
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

dt=0.01
T=0.02
for nx in [25, 50, 100,200, 400]:
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

dt = 0.01
T = 0.5
for nx in [25, 50, 100,200, 400]:
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

* let u_D = 0.0: made no difference in the errors

* using different stabilization code:

    r = u - u_n + dt*(- mu*div(grad(u)) + dot(velocity, grad(u)) + sigma*u - f)
    
    vnorm = sqrt(dot(velocity, velocity))
    
    F += (h/(2.0*vnorm))*dot(velocity, grad(v))*r*dx


dt=0.01
T=0.01
print("nx L2 H1 maxval minval")
for nx in [25, 50, 100, 200, 400]:
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

* let u_D = 0.0
dt = 0.01
T = 0.5
for nx in [25, 50, 100,200, 400]:
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

* putting in F-= new terms instead of F+= new terms

dt=0.01
T=0.01
print("nx L2 H1 maxval minval")
for nx in [25, 50, 100, 200, 400]:
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

dt=0.01
T=0.5
print("nx L2 H1 maxval minval")
for nx in [25, 50, 100, 200, 400]:
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

* multiplying dt to stabilization term!
* results below use corrected code

dt=0.01
T=0.01
print("nx L2 H1 maxval minval")
for nx in [25, 50, 100, 200, 400]:
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

In [3]:
dt=0.01
T=0.5
print("nx L2 H1 maxval minval")
for nx in [25, 50, 100, 200, 400]:
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

nx L2 H1 maxval minval
25 , 0.232984120591 , 16.6556011456 , 1.7299274489 , -1.17316411806
50 , 0.126601933786 , 11.9688731645 , 1.35308931215 , -0.526353278572
100 , 0.0230433899242 , 5.41872672781 , 1.052312783 , -0.12932188261
200 , 0.00453067404374 , 1.82727204769 , 0.991659653537 , -0.0218717263776
400 , 0.00214800409935 , 0.287680072835 , 0.98897505186 , -0.00457022855774
